<center>

<h1 style="text-align:center"> Modules and Functional Data Structures </h1>
<h2 style="text-align:center"> CSCI7000-11 S23: Principles of Functional Programming </h2>
</center>

In [1]:
module type COMPARABLE = sig
  type t
  val compare: t -> t -> int
end

module type COMPARABLE = sig type t val compare : t -> t -> int end


In [2]:
module type SET = sig
  type t
  module S:COMPARABLE
  val empty: unit -> t
  val insert: t -> S.t -> t
  val is_empty: t -> bool
  val exists: t -> S.t -> bool
end

module type SET =
  sig
    type t
    module S : COMPARABLE
    val empty : unit -> t
    val insert : t -> S.t -> t
    val is_empty : t -> bool
    val exists : t -> S.t -> bool
  end


In [7]:
module BST(S:COMPARABLE): SET = struct
  module S = S

  type t = 
    | Leaf
    | Branch of t * S.t * t
    
  let empty () = Leaf
    
  let rec insert t x = match t with
   | Leaf -> Branch(Leaf, x, Leaf)
   | Branch (lt, y, rt) -> 
       if S.compare x y <= 0 then Branch(insert lt x, y, rt)
       else Branch(lt, y, insert rt x)
       
  let is_empty t = match t with
   | Leaf -> true
   | _ -> false
   
  let rec exists t x  = failwith "not impl."
end

module BST : functor (S : COMPARABLE) -> SET


In [8]:
let rec length = function
  | [] -> 0
  | h::t -> 1 + (length t)

val length : 'a list -> int = <fun>


In [ ]:
module RBTree(S:COMPARABLE):SET = struct

  module S = S
  
  (*
    RedBlack tree is a binary search tree with two additional invariants:
     1. A Red node cannot be a child of another Red node
     2. The number of black nodes on all paths is equal
  *)
  
  type color = Red | Black
  
  type t =
    | E
    | T of color * {lt:t} * S.t * {rt:t| black_height(rt) = black_height(lt)}
    
  let empty () = E
  
  (*
    a -> x -> b -> y -> c -> z -> d
  *)
  let balance = function
   | (R, lt, x, rt) -> T (R, lt, x, rt)
   | (B,(R,(R,a,x,b), y,c),z,d) 
   | (B,(R,a,x,(R,b,y,c)),z,d) 
   | (B,a,x,(R,(R,b,y,c),z,d))
   | _ 
   -> T (B, (R,a,x,b),y,(R,c,z,d))
  
  let rec ins t x = match t with
    | E -> T (Red, E, x, E)
    | T (c,lt,y,rt) ->  
            if S.compare x y <= 0 
            then balance (c,ins lt x, y, rt)
            else balance (c, lt, y, ins rt x)
       
  let insert t x = match ins t x with
    | E -> E
    | T (_,lt,y,rt) -> T (Black, lt, y, rt)
end

In [9]:
(*

  streams 

*)
let rec ones = 1::ones

val ones : int list = [1; <cycle>]


In [10]:
ones

- : int list = [1; <cycle>]


In [11]:
let k = List.map (fun i -> i+1) ones 

error: runtime_error

In [12]:
type 'a stream = Cons of 'a * 'a stream

type 'a stream = Cons of 'a * 'a stream


In [13]:
let rec from n = Cons (n, from (n+1))

val from : int -> int stream = <fun>


In [14]:
from 10

error: runtime_error

In [16]:
match from 20 with
 | Cons(x,_) -> x

error: runtime_error

In [17]:
raise (Invalid_argument "blah")

error: runtime_error

In [18]:
(*
  Thunk
*)  
fun () -> raise (Invalid_argument "blah")

- : unit -> 'a = <fun>


In [19]:
type 'a stream = Cons of 'a * (unit -> 'a stream)

type 'a stream = Cons of 'a * (unit -> 'a stream)


In [20]:
let rec from n = Cons (n, fun () -> from (n+1))

val from : int -> int stream = <fun>


In [22]:
match from 20 with
 | Cons(x,f) -> (x,f())

- : int * int stream = (20, Cons (21, <fun>))
